# Collect Sample

In [ ]:
import cv2
import numpy as np

In [ ]:
model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Sample image collection
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = model.detectMultiScale(gray, 1.3, 5)
    if faces == ():
        return None
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face

In [ ]:
def sample(path): 
    # Open Webcam
    cap = cv2.VideoCapture(0)
    count = 0
    while True:
        ret , photo = cap.read()
        if face_extractor(photo) is not None:
            count+=1
            face = cv2.resize(face_extractor(photo), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = path + str(count) + '.jpg'
            cv2.imwrite(file_name_path, face)

            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Cropper', face)
        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
            break 
    cap.release()
    cv2.destroyAllWindows()
    print("##############################")
    print("Sample Collected...")
    print("##############################")

In [ ]:
#collect sample of person1
sample('./faces/Person1/')

In [ ]:
#collect sample of person2
sample('./faces/Person2/')

# Train Model

In [ ]:
#pip install --user opencv-contrib-python --upgrade

In [ ]:
#import cv2
#import numpy as np
from os import listdir
from os.path import isfile, join

In [ ]:
def train_model(path, name):
    data_path = path
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
    Training_Data, Labels = [], []
    for i, files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[i]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        Labels.append(i)

    Labels = np.asarray(Labels, dtype=np.int32)
    model  = cv2.face_LBPHFaceRecognizer.create()

    model.train(np.asarray(Training_Data), np.asarray(Labels))
    print("##############################")
    print("Model trained sucessefully")
    print("##############################")
    model.save('./Saved_model/{}_model.xml'.format(name))# save the trained model

In [ ]:
# train and save model of person1
train_model('./faces/Person1/', 'Person1')

In [ ]:
# train and save model of person2
train_model('./faces/Person2/', 'Person2')

# Recognise Face
## When it recognizes your face then - 
### 👉 It sends mail to your mail-id by writing this is the face of your_name.
### 👉 Second, it sends a WhatsApp message to your friend.

## When it recognizes a second face then -
### 👉 Create EC2 instance in the AWS and 
### 👉 Create 5 GB EBS volume and attach it to the instance using Terraform 

In [ ]:
#import cv2
#import numpy as np
import os
import msg


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

def Face_Recognision(model, name):
    # Open Webcam
    cap = cv2.VideoCapture(0)

    while True:

        ret, frame = cap.read()

        image, face = face_detector(frame)

        try:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

                # Pass face to prediction model
                # "results" comprises of a tuple containing the label and the confidence value
            results = model.predict(face)

            if results[1] < 500:
                confidence = int( 100 * (1 - (results[1])/400) )
                display_string = str(confidence) + '% Confident it is User'

            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)

            if confidence >= 90 and name == 'Person1':
                cv2.putText(image, "Hey {}".format(name), (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow('Face Recognition', image )
                cv2.imwrite("pic.jpg", face)
                msg.send_mail(name, 'pic.jpg')
                msg.send_watsapp_msg(name)
                break


            elif confidence >= 90 and name == "Person2":
                cv2.putText(image, "Hey {}".format(name), (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow('Face Recognition', image )
                msg.aws()
                break


            else:
                cv2.putText(image, "Face Not Recognised", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                cv2.imshow('Face Recognition', image )

        except:
            cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            pass

        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break

    cap.release()
    cv2.destroyAllWindows()     

In [ ]:
#person1
Person1_model = cv2.face_LBPHFaceRecognizer.create()
Person1_model.read('./Saved_model/Person1_model.xml')
Face_Recognision(Person1_model, 'Person1')

In [ ]:
#person2
Person2_model = cv2.face_LBPHFaceRecognizer.create()
Person2_model.read('./Saved_model/Person2_model.xml')
Face_Recognision(Person2_model, 'Person2')